## Практикум 3.1. ОПТИМИЗАЦИЯ ПАРАМЕТРОВ НЕЙРОСЕТЕВОГО АВТОЭНКОДЕРА

**Тип работы**:  Индивидуальная работа
Даны исходные данные. Требуется подобрать параметры нейросети и параметры обучения нейросети, чтобы обеспечить определенную точность на контрольных данных.

Для этого необходимо изменить две функции:

create_neuro_ae(nr, nc, encoding_dim) - описание сжимающего автоэнкодера для кодирования изображений размером nr*nc  кодом длины encoding_dim из двух полносвязных слоев: енкодера и декодера;

learn_neuro_ae(x_train, x_test) - функция создания и обучения нейросети автоенкодера с использованием обучающей и тестовой выборок (x_train, x_test);

так, чтобы ошибка восстановления изображения (потери - loss) и точность восстановления (accuracy) отличались бы от базовых значений (определяемых нейросетью преподавателя) меньше, чем на заданную точность. Для оценки кода и начисления баллов используются 4 теста на простых черно-белых изображениях цифр (digits.csv) размером 10*7 и 3 теста на массиве "серых" цифр размера 15*15 (digits_15x15.csv). Тесты отличаются требованиям по возможному отклонению от показателей преподавательской нейросети.

In [ ]:
# Модуль создания и обучения нейросетевого автоэнкодера
import numpy as np
import pandas as pd
import random
from keras.layers import Input, Dense, Flatten, Reshape
from keras.models import Model

# создадим сжимающий автоэнкодер с кодом малой размерности
# для кодирования изображений размером nr*nc из двух полносвязных слоев: енкодера и декодера.
# Так как интенсивность цвета нормирована на единицу, то активацию выходного слоя возьмем сигмоидой.
# Напишем отдельные модели для энкодера, декодера и целого автоэнкодера.
# Для этого создадим экземпляры слоев и применим их один за другим, в конце все объединив в модели.
# nr, nc, 1 - размерности строк, столбцов, фильтров одной картинки, без батч-размерности
# encoding_dim - Размерность кодированного представления
def create_neuro_ae(nr, nc, encoding_dim=10):
    # Энкодер
    # Входной плейсхолдер
    input_img = Input(shape=(nr, nc, 1))
    # Вспомогательный слой решейпинга
    flat_img = Flatten()(input_img)
    # Кодированное полносвязным слоем представление
    encoded = Dense(encoding_dim, activation='relu')(flat_img)

    # Декодер
    # Раскодированное другим полносвязным слоем изображение
    input_encoded = Input(shape=(encoding_dim,))
    flat_decoded = Dense(nc * nr, activation='sigmoid')(input_encoded)
    decoded = Reshape((nr, nc, 1))(flat_decoded)

    # Модели, в конструктор первым аргументом передаются входные слои, а вторым выходные слои
    # Другие модели можно так же использовать как и слои
    encoder = Model(input_img, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")
    return encoder, decoder, autoencoder

# Создадим и скомпилируем модель (под компиляцией в данном случае понимается построение графа вычислений обратного распространения ошибки)
# --- Эту функцию программирует обучающийся!!! ----
# функция создания и обучения нейросети автоенкодера
# x_train, x_test - выборки для обучения и валидации модели
def learn_neuro_ae(x_train, x_test):
    # определяем размеры матрицы описания одного объекта
    nr = x_train.shape[1]
    nc = x_train.shape[2]
    # --- Выбираем размерность кодированного представления  ! ---
    encoding_dim = 10
    # --- Выбираем параметры обучения                       ! ---
    n_epoh = 50
    batchsize = 10
    # -------- описываем нейросеть - создаем модель -----------
    encoder, decoder, ae_nnet = create_neuro_ae(nr= nr, nc= nc, encoding_dim= encoding_dim)

    # -------- комплируем нейросеть - выбираемм алгоритм обучения !
    ae_nnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

    # обучаем нейросеть - выбираем кол-во эпох, размер пакета
    ae_nnet.fit(x_train, x_train,
                epochs=n_epoh,
                batch_size=batchsize,
                shuffle=True,
                validation_data=(x_test, x_test))
    return ae_nnet
